In [28]:
import pandas as pd
from pathlib import Path

This script will add a column for all classmates of the students. Classmates are placed in random order as a comma separated string. 

In [29]:
participant_data_path = "./example_data"
participant_input_filename = "test_participantList.csv"

random_seed=28102024

In [30]:
column_dict = {
    "Personal Data": "attribute_2 <PersonalData>",
    "ClassId": "attribute_3 <ClassID>",
    "Stamnr": "token",
    "Name":"firstname"
}
classmates_column_name = "attribute_1 <Classmates>"
pupils_column_name = "attribute_1 <Pupils>"

In [31]:
participant_data_path = Path(participant_data_path)
participant_input_file = participant_data_path / participant_input_filename
school_df = pd.read_csv(participant_input_file)
school_df = school_df.loc[school_df["Consent"] != "No"].drop(["School", "Class", "Gender", "Age", "Consent"], axis=1)

## Students file preparation

In [32]:
students_df = school_df.rename(columns=column_dict).drop("Mentor", axis=1)

In [33]:
for i, student in students_df.iterrows():
    students_df.at[i,classmates_column_name] = students_df \
        .drop(i) \
        .loc[students_df["attribute_3 <ClassID>"] == student["attribute_3 <ClassID>"]] \
        .firstname \
        .sample(frac=1, random_state=random_seed).str.cat(sep=', ')

In [34]:
students_df["language"] = "nl"
students_df["attribute_2 <PersonalData>"] = students_df["attribute_2 <PersonalData>"].fillna("true").replace("No", "false")

In [35]:
students_df

,Grade,attribute_3 <ClassID>,Nr,firstname,token,LongId,attribute_2 <PersonalData>,attribute_1 <Classmates>,language
0,1,1,1,Harold,123451,999990001,true,Fatima,nl
2,1,1,3,Fatima,123453,999990003,true,Harold,nl
3,2,2,1,Charissa,123454,999990004,false,"Abdul, Derek, Dwayne",nl
4,2,2,2,Dwayne,123455,999990005,false,"Abdul, Derek, Charissa",nl
5,2,2,3,Abdul,123456,999990006,true,"Dwayne, Derek, Charissa",nl
6,2,2,4,Derek,123457,999990007,true,"Dwayne, Abdul, Charissa",nl


In [36]:
students_output_file = participant_input_file.with_stem(participant_input_file.stem + "_LimeSurveyStudents")
print(students_output_file)
students_df.to_csv(students_output_file)

example_data/test_participantList_LimeSurveyStudents.csv


## Mentors file preparation

In [37]:
mentor_list = [j for k in [i.split('/') for i in school_df["Mentor"].unique()] for j in k]
mentors_df = pd.DataFrame({"firstname":mentor_list})
for i, mentor in mentors_df.iterrows():
    mentors_df.at[i,pupils_column_name] = students_df[school_df["Mentor"].str.contains(mentor["firstname"])] \
        .firstname \
        .sample(frac=1).str.cat(sep=', ')

In [38]:
mentors_df["language"] = "nl"
mentors_df

,firstname,attribute_1 <Pupils>,language
0,Mister Something,"Fatima, Harold",nl
1,Mrs Else,"Harold, Fatima",nl
2,Mister This,"Derek, Abdul, Charissa, Dwayne",nl
3,Mrs That,"Dwayne, Abdul, Charissa, Derek",nl


In [39]:
mentors_output_file = participant_input_file.with_stem(participant_input_file.stem + "_LimeSurveyMentors")
print(mentors_output_file)
mentors_df.to_csv(mentors_output_file)

example_data/test_participantList_LimeSurveyMentors.csv
